# Explore here

In [81]:
#Los IMPORTS

import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

import regex as re

from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from IPython.display import display

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV
from pickle import dump




In [50]:
# Importar CSV
df = pd.read_csv("https://breathecode.herokuapp.com/asset/internal-link?id=932&path=url_spam.csv")
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [51]:
#Dimensiones del DataFrame
df.shape

(2999, 2)

In [52]:
#Resumen completo del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [53]:
#Eliminar duplicados del DataFrame
df = df.drop_duplicates()

df.shape

(2369, 2)

In [54]:
#Convertir "is_spam" a numérico
df["is_spam"] = df["is_spam"].astype(int)

df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [55]:
#Mostrar el total de valores en "spam" y "no spam"
print(df.shape)
print(f"Hay {len(df.loc[df.is_spam == 1])} correos que son spam")
print(f"Hay {len(df.loc[df.is_spam == 0])} correos que no son spam")

(2369, 2)
Hay 244 correos que son spam
Hay 2125 correos que no son spam


In [56]:
#Procesamos texto y eliminamos carácteres no deseados, reducción de espacios, dividir palabras
def preprocess_text(text):
    text = re.sub(r"[^a-z0-9 ]", " ", text) #Reemplazar carácteres que no sean letra (a-z) / dígito (0-9) / espacio con un espacio.
    text = re.sub(r"\b[a-zA-Z]\b", "", text) #Eliminar letras individuales
    text = re.sub(r"\s+", " ", text.lower()) #Reducción de multiples espacios a un espacio / convertir téxto a minúscula
    text = re.sub(r"&lt;/?[^\s]*?&gt;", " ", text) #Eliminar tags HTML
    text = text.strip() #Eliminar espacios al inicio y al final
    return text.split()

df["url_procesada"] = df["url"].apply(preprocess_text) #Preprocesar "url" y pasar a "url procesada"
display(df.head())

,url,is_spam,url_procesada
0,https://briefingday.us8.list-manage.com/unsubs...,1,"[https, briefingday, us8, list, manage, com, u..."
1,https://www.hvper.com/,1,"[https, www, hvper, com]"
2,https://briefingday.com/m/v4n3i4f3,1,"[https, briefingday, com, v4n3i4f3]"
3,https://briefingday.com/n/20200618/m#commentform,0,"[https, briefingday, com, 20200618, commentform]"
4,https://briefingday.com/fan,1,"[https, briefingday, com, fan]"


In [57]:
# Descargar los recursos necesarios de NLTK si no están ya presentes
download("wordnet")
download("stopwords")
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words("english")

# Función para lematizar palabras y eliminar stopwords y palabras cortas
def lemmatize_text(words, lemmatizer = lemmatizer):
    # Lematizar cada palabra en la lista
    tokens = [lemmatizer.lemmatize(word) for word in words]
    # Eliminar stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Eliminar palabras con longitud menor o igual a 3 caracteres
    tokens = [word for word in tokens if len(word) > 3]
    return tokens

# Aplicar la función de lematización a la columna 'processed_url'
df["url_procesada"] = df["url_procesada"].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:

#Vectorizar texto para el modelo
lista_tokens = df["url_procesada"]
lista_tokens = [" ".join(tokens) for tokens in lista_tokens]
vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)

X = vectorizer.fit_transform(lista_tokens).toarray()
y = df["is_spam"]

print("Primeras 5 filas de la matriz de características X:")
display(X[:5])

print("\nMatriz de características X:", X.shape)
print("Variable objetivo y:", y.shape)

Primeras 5 filas de la matriz de características X:


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(5, 543))


Matriz de características X: (2369, 543)
Variable objetivo y: (2369,)


In [59]:
#Train / Test / Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print("Conjunto de entrenamiento X_train:", X_train.shape)
print("Conjunto de prueba X_test:", X_test.shape)
print("Conjunto de entrenamiento y_train:", y_train.shape)
print("Conjunto de prueba y_test:", y_test.shape)

Conjunto de entrenamiento X_train: (1895, 543)
Conjunto de prueba X_test: (474, 543)
Conjunto de entrenamiento y_train: (1895,)
Conjunto de prueba y_test: (474,)


In [60]:
#Inicialización de modelo y entrenamiento
model = SVC(kernel = "linear", random_state = 42)
model.fit(X_train, y_train)


,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [61]:
#Predicción del modelo
y_pred = model.predict(X_test)
y_pred


array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [77]:
#Revisión de precisión accuracy / precision / recall / F1 Score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"La precisión en (Accuracy) es de: {accuracy:.4f}")
print(f"La precisión en (Precision) es de: {precision:.4f}")
print(f"La precisión en (Recall) es de: {recall:.4f}")
print(f"La puntuación F1 (F1 Score): {f1:.4f}")

La precisión en (Accuracy) es de: 0.9578
La precisión en (Precision) es de: 0.8519
La precisión en (Recall) es de: 0.5897
La puntuación F1 (F1 Score): 0.6970


In [65]:
#Optimizar modelo SVM con Grid Search
param_grid = {"C": [0.1, 1, 10, 100], "kernel": ["linear", "rbf"], "gamma": ["scale", "auto"]}

print("La cuadrícula de hiperparámetros definida es:")
print(param_grid)


La cuadrícula de hiperparámetros definida es:
{'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}


In [69]:
#Buscando mejores hiperparámetros para el modelo con Grid Search
svm_model = SVC(random_state=42)
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring="f1", n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Mejores hiperparámetros encontrados")

Mejores hiperparámetros encontrados


In [76]:
#Mejores parámetros para el Grid Search
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_svm_model = grid_search.best_estimator_
print(best_svm_model)
print(f"Los mejores hiperparámetros son: {best_params}")
print("\nEl mejor modelo SVM entrenado con los mejores hiperparámetros es: ")
print("\nEl mejor score F1 es el:")
print(f"{best_score:.4f}")


SVC(C=10, random_state=42)
Los mejores hiperparámetros son: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

El mejor modelo SVM entrenado con los mejores hiperparámetros es: 

El mejor score F1 es el:
0.6208


In [80]:
#Evaluación del modelo optimizado
y_pred_optimizado = best_svm_model.predict(X_test)
accuracy_optimizado = accuracy_score(y_test, y_pred_optimizado)
precision_optimizado = precision_score(y_test, y_pred_optimizado)
recall_optimizado = recall_score(y_test, y_pred_optimizado)
f1_optimizado = f1_score(y_test, y_pred_optimizado)

print("Resultado para el modelo SVM ya optimizado:")
print(f"La precisión (Accuracy) es de: {accuracy_optimizado:.4f}")
print(f"La precisión (Precision) es de: {precision_optimizado:.4f}")
print(f"La precisión en (Recall) es de: {recall_optimizado:.4f}")
print(f"La puntuación F1 (F1 Score) es de: {f1_optimizado:.4f}")

Resultado para el modelo SVM ya optimizado:
La precisión (Accuracy) es de: 0.9557
La precisión (Precision) es de: 0.7647
La precisión en (Recall) es de: 0.6667
La puntuación F1 (F1 Score) es de: 0.7123


Modelo antes de optimizar:
La precisión en (Accuracy) es de: 0.9578
La precisión en (Precision) es de: 0.8519
La precisión en (Recall) es de: 0.5897
La puntuación F1 (F1 Score): 0.6970

In [82]:
#Guardar modelo
dump(model, open("svm_classifier_linear_42.sav", "wb"))

Después de la optimización mejora en accuracy un +.0021, en precision baja un -0,0872, en recall también cae en un -0.077 y sube en el F1 score un +0.0153... Aquí tendría que consultar si merece la pena optimizar siendo que dos valores bajan...